# Define Agents

In [ ]:
from langchain_ollama import ChatOllama
llm= ChatOllama(model="mradermacher/Llama-3.1-8B-UltraMedical-i1-GGU", temperature=0)
messages = [HumanMessage(content="What are the different types of Diabetes")]
response = chat_model.invoke(messages)
print(response.content)

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
# We need this because we want to enable threads (conversations)
checkpointer = InMemorySaver()



In [ ]:
from langchain.embeddings import init_embeddings
from langgraph.store.memory import InMemoryStore
in_memory_store = InMemoryStore(
    index={
        "embed": init_embeddings("openai:text-embedding-3-small"),  # Embedding provider
        "dims": 1536,                              # Embedding dimensions
        "fields": ["food_preference", "$"]              # Fields to embed
    }
)

In [ ]:
from typing import TypedDict, Annotated, List
import operator
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Define the state for our agent
class HealthAgentState(TypedDict):
    messages: Annotated[List, operator.add]
    user_condition: str
    retrieved_info: str
    recommendation: dict
    

In [ ]:
# Setup MedlinePlus retrieval system
def setup_medline_retriever():
    """
    Setup retriever with MedlinePlus health topics
    In production, you would load actual MedlinePlus data
    """
    # Example MedlinePlus topics URLs (add more as needed)
    medline_urls = [
        "https://medlineplus.gov/diabetes.html",
        "https://medlineplus.gov/highbloodpressure.html",
        "https://medlineplus.gov/heartdiseases.html",
        "https://medlineplus.gov/asthma.html",
        "https://medlineplus.gov/anxiety.html",
    ]
    
    # Load documents
    loader = WebBaseLoader(medline_urls)
    docs = loader.load()
    
    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(docs)
    
    # Create vector store
    vectorstore = FAISS.from_documents(splits, OpenAIEmbeddings())
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    
    return retriever

# Initialize retriever (do this once at startup)
retriever = setup_medline_retriever()

# Node functions
def extract_condition(state: HealthAgentState) -> HealthAgentState:
    """Extract and understand the user's health condition"""
    messages = state["messages"]
    user_input = messages[-1].content if messages else ""
    
    # Use LLM to extract key health information
    extraction_prompt = f"""
    Extract the main health condition or symptoms from this user input:
    "{user_input}"
    
    Provide a concise summary of the health condition.
    """
    
    response = llm.invoke([HumanMessage(content=extraction_prompt)])
    
    return {
        "user_condition": response.content,
        "messages": [AIMessage(content=f"Understanding your condition: {response.content}")]
    }

def retrieve_information(state: HealthAgentState) -> HealthAgentState:
    """Retrieve relevant health information from MedlinePlus"""
    condition = state["user_condition"]
    
    # Retrieve relevant documents
    docs = retriever.get_relevant_documents(condition)
    
    # Combine retrieved information
    retrieved_text = "\n\n".join([doc.page_content for doc in docs])
    
    return {
        "retrieved_info": retrieved_text,
        "messages": [AIMessage(content="Retrieved relevant medical information...")]
    }

In [ ]:
def generate_recommendation(state: HealthAgentState) -> HealthAgentState:
    """Generate health recommendations based on retrieved information"""
    condition = state["user_condition"]
    retrieved_info = state["retrieved_info"]
    
    recommendation_prompt = f"""
    Based on the following health condition and medical information, provide recommendations.
    
    User's Condition: {condition}
    
    Medical Information from MedlinePlus:
    {retrieved_info}
    
    Provide a structured recommendation with the following categories:
    1. Should Consult Doctor: (Yes/No and urgency level)
    2. Recommended Tests: (List specific tests if applicable)
    3. Lifestyle Changes: (Specific actionable recommendations)
    4. Warning Signs: (Symptoms that require immediate attention)
    5. Self-Care Tips: (What they can do at home)
    
    Be specific, actionable, and balanced. Always err on the side of caution.
    Format as a clear, structured response.
    """
    
    response = llm.invoke([
        SystemMessage(content="You are a health information assistant. Provide clear, evidence-based recommendations while emphasizing when professional medical advice is needed."),
        HumanMessage(content=recommendation_prompt)
    ])
    
    # Parse the recommendation (simplified version)
    recommendation_dict = {
        "full_text": response.content,
        "consult_doctor": "should consult" in response.content.lower() or "see a doctor" in response.content.lower(),
        "urgency": "immediate" in response.content.lower() or "urgent" in response.content.lower()
    }
    
    return {
        "recommendation": recommendation_dict,
        "messages": [AIMessage(content=response.content)]
    }

def format_response(state: HealthAgentState) -> HealthAgentState:
    """Format the final response to the user"""
    recommendation = state["recommendation"]
    
    disclaimer = "\n\n⚠️ IMPORTANT DISCLAIMER: This is for informational purposes only and not a substitute for professional medical advice. Always consult with a qualified healthcare provider for proper diagnosis and treatment."
    
    final_message = recommendation["full_text"] + disclaimer
    
    return {
        "messages": [AIMessage(content=final_message)]
    }


In [ ]:
# Build the graph
def create_health_agent():
    """Create the LangGraph health assessment agent"""
    workflow = StateGraph(HealthAgentState)
    
    # Add nodes
    workflow.add_node("extract_condition", extract_condition)
    workflow.add_node("retrieve_information", retrieve_information)
    workflow.add_node("generate_recommendation", generate_recommendation)
    workflow.add_node("format_response", format_response)
    
    # Add edges
    workflow.set_entry_point("extract_condition")
    workflow.add_edge("extract_condition", "retrieve_information")
    workflow.add_edge("retrieve_information", "generate_recommendation")
    workflow.add_edge("generate_recommendation", "format_response")
    workflow.add_edge("format_response", END)
    
    # Compile the graph with the checkpointer and store
    graph = workflow.compile(checkpointer=checkpointer, store=in_memory_store)
    
    return graph